In [1]:
import os
os.chdir("..")

In [2]:
import json, requests

In [3]:
import pandas as pd
import numpy as np

from src.scrub import get_clean_iris
from src.deploy import generate_random_iris
from src.deploy import get_prediction

# Stream Prediction
---

- define the url where the api is served
- set up sample payload
- send a `post` request with the payload to the api
- decode the returned json into a dict
- change the payload and re-run a few times

In [4]:
url = "http://127.0.0.1:5001/api"

In [5]:
payload = json.dumps({
    'sl': 3,
    'sw': 5,
    'pl': 2.75,
    'pw': 2.1,
    'algo': 'dt'
})

In [6]:
r = requests.post(url, payload)

In [7]:
json.loads(r.text)

{u'iris_type': u'versicolor', u'model': u'dt'}

# Batch Predictions
---

- we simulate some real-world data by adding noise to actual data
- rename columns so they are consistent with what the api expects
- define a function to retrieve a prediction
- map the function over the rows

In [16]:
df = generate_random_iris()

In [35]:
test_case = df.sample(1).iloc[0]
print(test_case)

get_prediction(test_case, 'dt', url=url)

sl    7.45
sw    4.81
pl    5.11
pw    1.17
Name: 0.75, dtype: float64


{u'iris_type': u'virginica', u'model': u'dt'}

In [36]:
df['predicted_iris_type'] = \
(df
 .apply(lambda r: get_prediction(r, 'lr', url=url).get('iris_type'), 
        axis=1)
)

In [39]:
df.head()

,sl,sw,pl,pw,predicted_iris_type
0.75,7.45,4.81,5.11,1.17,versicolor
0.75,5.88,1.78,4.02,0.94,versicolor
0.75,5.87,3.53,3.58,0.46,versicolor
0.75,6.67,2.83,4.59,1.74,versicolor
0.75,6.06,3.17,5.18,1.28,versicolor
